In [1]:
import random
from transformers import pipeline

# Inicjalizacja modelu (np. GPT-2)
model = pipeline("text-generation", model="gpt2", pad_token_id=50256)

random.seed(42)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
# Definicja funkcji do testowania wyrażeń arytmetycznych
def test_model(prompt, true_answer):
    # Generowanie odpowiedzi modelu
    output = model(prompt, max_new_tokens=2, num_return_sequences=1)
    response = output[0]["generated_text"].split("=")[-1].strip()
    response = "".join([char for char in response if char.isnumeric()])
    # Sprawdzenie poprawności
    # print(f"response = {response}, true_value = {true_answer}")

    try:
        model_answer = int(response)
        return model_answer == true_answer
    except ValueError as error:
        print(str(error))
        return False  # Jeśli model nie zwróci liczby


def read_pairs_from_file(filename):
    pairs = []
    with open(filename, "r") as file:
        for line in file:
            # Rozdzielenie linii na wyrażenie i wynik
            expression, result = line.strip().split("=")
            # Usunięcie ewentualnych spacji i konwersja wyniku na liczbę całkowitą
            expression = expression.strip() + " ="
            result = int(result.strip())
            # Dodanie krotki do listy
            pairs.append((expression, result))
    return pairs


print(read_pairs_from_file("data/addition_lines.txt"))

[('1 + 1 =', 2), ('2 + 2 =', 4), ('3 + 3 =', 6), ('4 + 4 =', 8), ('5 + 5 =', 10), ('6 + 6 =', 12), ('7 + 7 =', 14), ('8 + 8 =', 16), ('9 + 9 =', 18), ('10 + 10 =', 20), ('11 + 11 =', 22), ('12 + 12 =', 24), ('13 + 13 =', 26), ('14 + 14 =', 28), ('15 + 15 =', 30), ('16 + 16 =', 32), ('17 + 17 =', 34), ('18 + 18 =', 36), ('19 + 19 =', 38), ('20 + 20 =', 40), ('21 + 21 =', 42), ('22 + 22 =', 44), ('3 + 3 =', 6), ('3 + 4 =', 7), ('3 + 3 =', 6), ('3 + 7 =', 10), ('7 + 10 =', 17), ('3 + 4 =', 7), ('6 + 10 =', 16), ('1 + 4 =', 5), ('10 + 1 =', 11)]


In [14]:
# Testowanie modeli na różnych wyrażeniach
test_data_paths = [
    "data/addition_lines.txt",
    "data/substraction_lines.txt",
    "data/multiplication_lines.txt",
    "data/division_lines.txt",
]

positives = {"addition": [], "substraction": [], "multiplication": [], "division": []}

for path in test_data_paths:
    operation = path.replace("data/", "").replace("_lines.txt", "")

    data = read_pairs_from_file(path)
    random.shuffle(data)
    data = data[:50]
    split_index = int(len(data) * 0.4)
    few_shot_learning_data = data[:split_index]
    # print(few_shot_learning_data)
    examples = data[split_index:]
    # print("examples:", examples)
    few_shot_prompt = ""

    for ecuation, result in few_shot_learning_data:
        few_shot_prompt += f"{ecuation}{result}\n"

    results = []

    for expr, answer in examples:  # show for [:10]
        prompt = few_shot_prompt + expr
        # print(prompt)
        result = test_model(prompt, answer)
        results.append(result)
        if result:
            positives[operation].append(expr)

    # Analiza wyników
    accuracy = sum(results) / len(results)
    print(f"Model Accuracy for {operation}:", round(accuracy, 2) * 100, "%")

Model Accuracy for addition: 5.0 %
Model Accuracy for substraction: 6.0 %
Model Accuracy for multiplication: 0.0 %
Model Accuracy for division: 17.0 %


In [15]:
print(positives["addition"])
print(positives["substraction"])
print(positives["multiplication"])
print(positives["division"])

['2 + 2 =']
['60 - 6 =']
[]
['120 / 30 =', '40 / 8 =', '96 / 24 =']


<!-- @format -->

# Wnioski

1. Dzielenie (17%): Model radzi sobie tutaj najlepiej spośród wszystkich operacji. Być może wynika to z tego, że dzielenie często zwraca liczby całkowite dla prostych wartości (jak 4 / 2 = 2). Operacje dzielenia, które generują pełne liczby, mogą być bardziej przewidywalne, a model może "rozpoznawać" wzorce liczbowe, które są zgodne z tymi wartościami.
2. Odejmowanie (5%) i Dodawanie (6): Są one podobne pod względem skuteczności, ale wyraźnie niższe niż dzielenie. Być może model ma większą trudność z tymi operacjami, ponieważ różnice liczbowe są bardziej zróżnicowane i mogą być trudniejsze do „zgadnięcia” na podstawie kilku przykładów.
3. Mnożenie (0%): Wynik jest najniższy. To może być efektem złożoności operacji – mnożenie może generować większe liczby, a zakres wyników jest bardziej rozproszony, co utrudnia modelowi trafne odpowiedzi w kontekście few-shot learningu.
